### import datasets from drive

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

# Change to the directory where your datasets are stored
data_dir = '/content/drive/My Drive/NEST'
os.chdir(data_dir)

# List files in the directory
!ls


df_eligibilities.csv  df_usecase_3.csv	 usecase_1_.csv			usecase_4_.csv
df_usecase_1.csv      df_usecase_4.csv	 usecase_2_TEST_gt_removed.csv
df_usecase_2.csv      eligibilities.csv  usecase_3_.csv


In [3]:
import pandas as pd

# Load datasets
df_usecase_2 = pd.read_csv('usecase_2_TEST_gt_removed.csv', low_memory=False)
df_usecase_1 = pd.read_csv('usecase_1_.csv', low_memory=False)
df_usecase_3 = pd.read_csv('usecase_3_.csv', low_memory=False)
df_usecase_4 = pd.read_csv('usecase_4_.csv', low_memory=False)
df_eligibilities = pd.read_csv('df_eligibilities.csv', encoding='ISO-8859-1', on_bad_lines='skip')

#dropping unknown columns
df_usecase_2 = df_usecase_2.iloc[:, 1:25]
df_usecase_1 = df_usecase_1.iloc[:, 2:31]
df_usecase_3 = df_usecase_3.iloc[:, 2:31]
df_usecase_4 = df_usecase_4.iloc[:, :28]

df_eligibilities.rename(columns={'Nct number': 'NCT Number'}, inplace=True)  # Rename 'Nct_id' to 'NCT Number'
df_eligibilities.rename(columns={'Id': 'Other IDs'}, inplace=True) # Rename the 'IDs' column to 'Other IDs'

# Print basic info about the datasets
print("Dataset 1 (usecase_2_TEST_gt_removed.csv):")
print(df_usecase_2.info())

print("\nDataset 2 (usecase_1_.csv):")
print(df_usecase_1.info())

print("\nDataset 3 (usecase_3_.csv):")
print(df_usecase_3.info())

print("\nDataset 4 (usecase_4_.csv):")
print(df_usecase_4.info())

print("\nDataset 5 (eligibilities.csv):")
print(df_eligibilities.info())


Dataset 1 (usecase_2_TEST_gt_removed.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17286 entries, 0 to 17285
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   NCT Number                  17278 non-null  object
 1   Study Title                 17278 non-null  object
 2   Study URL                   17278 non-null  object
 3   Acronym                     4278 non-null   object
 4   Study Status                17277 non-null  object
 5   Brief Summary               17276 non-null  object
 6   Study Results               17276 non-null  object
 7   Conditions                  17277 non-null  object
 8   Interventions               17275 non-null  object
 9   Primary Outcome Measures    17276 non-null  object
 10  Secondary Outcome Measures  13145 non-null  object
 11  Other Outcome Measures      1724 non-null   object
 12  Sponsor                     17248 non-null  object
 13  Col

In [ ]:
# Define a function to derive age group from minimum_age and maximum_age of eligibilities

def get_age_group(min_age, max_age):
    # Convert minimum and maximum ages to numeric (in years)
    min_age = int(min_age.split()[0]) if isinstance(min_age, str) and min_age != 'N/A' else None
    max_age = int(max_age.split()[0]) if isinstance(max_age, str) and max_age != 'N/A' else None

    # Logic for determining age group
    if max_age and max_age < 18:
        return 'child'
    elif min_age and min_age >= 65:
        return 'older_adult'
    elif min_age and min_age >= 18 and (not max_age or max_age < 65):
        return 'adult'
    else:
        return 'Unknown'

# Apply the function to the dataset
df_eligibilities['age_group'] = df_eligibilities.apply(
    lambda row: get_age_group(row['minimum_age'], row['maximum_age']), axis=1
)

# Drop the original columns if no longer needed
df_eligibilities.drop(columns=['minimum_age', 'maximum_age', 'adult', 'child', 'older_adult'], inplace=True)

# Check the updated dataset
print(df_eligibilities['age_group'].value_counts())
# print(df_eligibilities.head())



age_group
adult          261567
Unknown        207509
child           25354
older_adult      7753
Name: count, dtype: int64


In [5]:
# change column name
df_usecase_1.rename(columns={'Age': 'Age_group'}, inplace=True)
df_usecase_2.rename(columns={'Age': 'Age_group'}, inplace=True)
df_usecase_3.rename(columns={'Age': 'Age_group'}, inplace=True)
df_usecase_4.rename(columns={'Age': 'Age_group'}, inplace=True)

In [6]:
# Check the unique values before normalization
print(df_usecase_1['Age_group'].value_counts())
print(df_usecase_2['Age_group'].value_counts())
print(df_usecase_3['Age_group'].value_counts())
print(df_usecase_4['Age_group'].value_counts())

Age_group
ADULT, OLDER_ADULT           81877
ADULT                        18341
CHILD, ADULT, OLDER_ADULT     9311
CHILD                         5551
CHILD, ADULT                  2207
OLDER_ADULT                    693
Name: count, dtype: int64
Age_group
ADULT, OLDER_ADULT                                                                                                                                                                                                                                                                                                                                                                                                                    10500
ADULT                                                                                                                                                                                                                                                                                                                       

In [7]:
def normalize_age_group(age_group):
    if pd.isnull(age_group) or age_group.strip() == "":
        return "unknown"

    # Convert to lowercase and split by comma
    age_groups = [group.strip().lower() for group in age_group.split(",")]

    # Define priority: child > adult > older_adult
    if "child" in age_groups:
        return "child"
    elif "adult" in age_groups:
        return "adult"
    elif "older_adult" in age_groups:
        return "older_adult"
    else:
        return "unknown"

# Apply normalization
df_usecase_1['Age_group'] = df_usecase_1['Age_group'].apply(normalize_age_group)
df_usecase_2['Age_group'] = df_usecase_2['Age_group'].apply(normalize_age_group)
df_usecase_3['Age_group'] = df_usecase_3['Age_group'].apply(normalize_age_group)
df_usecase_4['Age_group'] = df_usecase_4['Age_group'].apply(normalize_age_group)

# Check the unique values after normalization
print(df_usecase_1['Age_group'].value_counts())
print(df_usecase_2['Age_group'].value_counts())
print(df_usecase_3['Age_group'].value_counts())
print(df_usecase_4['Age_group'].value_counts())

Age_group
adult          100218
child           17069
older_adult       693
Name: count, dtype: int64
Age_group
adult          14113
child           2828
older_adult      272
unknown           73
Name: count, dtype: int64
Age_group
adult          203229
child           50522
older_adult      3826
Name: count, dtype: int64
Age_group
adult          18611
child           1981
older_adult       84
Name: count, dtype: int64


### drop irrelevant columns from the 5 different dataset files

In [8]:
# Verify and drop columns
drop_columns_2 = ['Acronym', 'Collaborators', 'Other Outcome Measures', 'Study Documents']
drop_columns_1 = ['Acronym', 'Collaborators', 'Other Outcome Measures', 'Results First Posted', 'First Posted', 'Last Update Posted', 'Start Date', 'Primary Completion Date', 'Completion Date']
drop_columns_3 = drop_columns_1
drop_columns_4 = ['Acronym', 'Collaborators', 'Other Outcome Measures', 'Results First Posted', 'Start Date', 'Primary Completion Date', 'Completion Date', 'First Posted', 'Last Update Posted']
# drop_columns_eligibilities = ['sampling_method', 'gender_description', 'gender_based', 'population']

# Drop columns with errors='ignore'
df_usecase_2.drop(columns=drop_columns_2, inplace=True, errors='ignore')
df_usecase_1.drop(columns=drop_columns_1, inplace=True, errors='ignore')
df_usecase_3.drop(columns=drop_columns_3, inplace=True, errors='ignore')
df_usecase_4.drop(columns=drop_columns_4, inplace=True, errors='ignore')
# df_eligibilities.drop(columns=drop_columns_eligibilities, inplace=True, errors='ignore')

In [9]:
# Print basic info about the datasets
print("Dataset 1 (usecase_2_TEST_gt_removed.csv):")
print(df_usecase_2.info())

print("\nDataset 2 (usecase_1_.csv):")
print(df_usecase_1.info())

print("\nDataset 3 (usecase_3_.csv):")
print(df_usecase_3.info())

print("\nDataset 4 (usecase_4_.csv):")
print(df_usecase_4.info())

print("\nDataset 5 (eligibilities.csv):")
print(df_eligibilities.info())


Dataset 1 (usecase_2_TEST_gt_removed.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17286 entries, 0 to 17285
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   NCT Number                  17278 non-null  object
 1   Study Title                 17278 non-null  object
 2   Study URL                   17278 non-null  object
 3   Study Status                17277 non-null  object
 4   Brief Summary               17276 non-null  object
 5   Study Results               17276 non-null  object
 6   Conditions                  17277 non-null  object
 7   Interventions               17275 non-null  object
 8   Primary Outcome Measures    17276 non-null  object
 9   Secondary Outcome Measures  13145 non-null  object
 10  Sponsor                     17248 non-null  object
 11  Sex                         17240 non-null  object
 12  Age_group                   17286 non-null  object
 13  Pha

### download individual 5 files after dropping irrelevant columns

In [10]:
# Save the 5 DataFrames
df_usecase_1.to_csv('df_usecase_1.csv', index=False)
print("Dataset saved as 'df_usecase_1.csv'")

df_usecase_2.to_csv('df_usecase_2.csv', index=False)
print("Dataset saved as 'df_usecase_2.csv'")

df_usecase_3.to_csv('df_usecase_3.csv', index=False)
print("Dataset saved as 'df_usecase_3.csv'")

df_usecase_4.to_csv('df_usecase_4.csv', index=False)
print("Dataset saved as 'df_usecase_4.csv'")

df_eligibilities.to_csv('df_eligibilities.csv', index=False)
print("Dataset saved as 'df_eligibilities.csv'")

Dataset saved as 'df_usecase_1.csv'
Dataset saved as 'df_usecase_2.csv'
Dataset saved as 'df_usecase_3.csv'
Dataset saved as 'df_usecase_4.csv'
Dataset saved as 'df_eligibilities.csv'


In [ ]:
df_eligibilities.columns = [col.capitalize() for col in df_eligibilities.columns]  # Capitalize all column namesId
df_eligibilities.rename(columns={'Nct number': 'NCT Number'}, inplace=True)  # Rename 'Nct_id' to 'NCT Number'
df_eligibilities.rename(columns={'Gender': 'Sex'}, inplace=True) # Rename the 'Gender' column to 'Sex'
df_eligibilities.rename(columns={'Other ids': 'Other IDs'}, inplace=True) # Rename the 'IDs' column to 'Other IDs'

### check duplicates

In [11]:
for df, name in zip([df_usecase_3, df_usecase_2, df_usecase_1, df_usecase_4, df_eligibilities],
                    ['usecase_3', 'usecase_2', 'usecase_1', 'usecase_4', 'eligibilities']):
    print(f"{name}: {df['NCT Number'].duplicated().sum()} duplicate rows")


usecase_3: 0 duplicate rows
usecase_2: 8 duplicate rows
usecase_1: 0 duplicate rows
usecase_4: 0 duplicate rows
eligibilities: 0 duplicate rows


In [12]:
print(df_usecase_2['NCT Number'].str.startswith('NCT').all())  # Should return True
print(df_usecase_2['NCT Number'].isna().sum())  # Check for missing values

False
8


In [13]:
# Drop rows where 'NCT Number' is missing or doesn't start with 'NCT'
df_usecase_2 = df_usecase_2[df_usecase_2['NCT Number'].str.startswith('NCT', na=False)]

In [14]:
print(df_usecase_2['NCT Number'].str.startswith('NCT').all())  # Should return True
print(df_usecase_2['NCT Number'].isna().sum())  # Should return 0


True
0


In [15]:
print("Unique NCT Numbers in usecase_3:", df_usecase_3['NCT Number'].nunique())
print("Unique NCT Numbers in usecase_2:", df_usecase_2['NCT Number'].nunique())
print("Unique NCT Numbers in usecase_1:", df_usecase_1['NCT Number'].nunique())
print("Unique NCT Numbers in usecase_4:", df_usecase_4['NCT Number'].nunique())
print("Unique NCT Numbers in eligibilities:", df_eligibilities['NCT Number'].nunique())

Unique NCT Numbers in usecase_3: 257577
Unique NCT Numbers in usecase_2: 17241
Unique NCT Numbers in usecase_1: 117980
Unique NCT Numbers in usecase_4: 20676
Unique NCT Numbers in eligibilities: 95772


In [16]:
common_nct = set(df_usecase_3['NCT Number'])
common_nct.intersection_update(df_usecase_2['NCT Number'])
common_nct.intersection_update(df_usecase_1['NCT Number'])
common_nct.intersection_update(df_usecase_4['NCT Number'])
common_nct.intersection_update(df_eligibilities['NCT Number'])
print("Number of common NCT Numbers across all datasets:", len(common_nct))

Number of common NCT Numbers across all datasets: 440


## Start merging df's

In [18]:
# Step-by-step merge analysis with logging
df_temp = df_usecase_3.copy()
print("Initial rows in usecase_3:", df_temp.shape[0])

# Merge with usecase_2
df_temp = pd.merge(df_temp, df_usecase_2, on='NCT Number', how='outer', suffixes=('', '_usecase2'))
print("After merging with usecase_2:", df_temp.shape[0])

# Merge with usecase_1
df_temp = pd.merge(df_temp, df_usecase_1, on='NCT Number', how='outer', suffixes=('', '_usecase1'))
print("After merging with usecase_1:", df_temp.shape[0])

# Merge with usecase_4
df_temp = pd.merge(df_temp, df_usecase_4, on='NCT Number', how='outer', suffixes=('', '_usecase4'))
print("After merging with usecase_4:", df_temp.shape[0])

# Merge with eligibilities
df_temp = pd.merge(df_temp, df_eligibilities, on='NCT Number', how='outer', suffixes=('', '_eligibilities'))
print("After merging with eligibilities:", df_temp.shape[0])

# Proceed with the final merge and cleanup
df_merged = df_temp.copy()

# Retain relevant columns
columns_to_keep = [
    'NCT Number', 'Study Title', 'Study URL', 'Study Status', 'Brief Summary',
    'Study Type', 'Study Results', 'Study Design', 'Sponsor', 'Conditions',
    'Interventions', 'Primary Outcome Measures', 'Secondary Outcome Measures',
    'Phases', 'Age_group', 'Sex', 'Healthy_volunteers', 'Criteria', 'Enrollment',
    'Locations', 'Other IDs'
]
df_merged = df_merged[[col for col in columns_to_keep if col in df_merged.columns]]

# Handle missing values
# Fill missing values in numeric columns with 0
numeric_columns = df_merged.select_dtypes(include=['float64', 'int64']).columns
df_merged[numeric_columns] = df_merged[numeric_columns].fillna(0)

# Fill missing values in non-numeric columns with 'unknown'
non_numeric_columns = df_merged.select_dtypes(include=['object']).columns
df_merged[non_numeric_columns] = df_merged[non_numeric_columns].fillna('unknown')

# Save the merged DataFrame
df_merged.to_csv('merged_clinical_trials_cleaned.csv', index=False)
print("Final merged dataset saved as 'merged_clinical_trials_cleaned.csv'")


Initial rows in usecase_3: 257577
After merging with usecase_2: 261172
After merging with usecase_1: 305130
After merging with usecase_4: 305884
After merging with eligibilities: 343676
Final merged dataset saved as 'merged_clinical_trials_cleaned.csv'


In [19]:
# Verify merged DataFrame
print("Merged DataFrame Info:")
print(df_merged.info())

Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343676 entries, 0 to 343675
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   NCT Number                  343676 non-null  object 
 1   Study Title                 343676 non-null  object 
 2   Study URL                   343676 non-null  object 
 3   Study Status                343676 non-null  object 
 4   Brief Summary               343676 non-null  object 
 5   Study Type                  343676 non-null  object 
 6   Study Results               343676 non-null  object 
 7   Study Design                343676 non-null  object 
 8   Sponsor                     343676 non-null  object 
 9   Conditions                  343676 non-null  object 
 10  Interventions               343676 non-null  object 
 11  Primary Outcome Measures    343676 non-null  object 
 12  Secondary Outcome Measures  343676 non-null  obje

In [20]:
# Check for duplicate NCT Numbers
print("Number of duplicate NCT Numbers:", df_merged.duplicated(subset='NCT Number').sum())

Number of duplicate NCT Numbers: 0
